# PyCity Schools Analysis

*This analysis uses district-wide standardized math and reading test results for every student.*

* Average reading scores decreased as spending per student increased. Schools with the highest budgets (>\$635.75 per student) had students with an average reading score of 81.368774, while schools with the lowest budgets (<\$597.25 per student) had students with an average reading score of 83.915256. 

* Average math scores generally decreased as spending per student increased. Schools with the highest budgets (>\$635.75 per student) had students with an average math score of 77.866721, while schools with the lowest budgets (<\$597.25 per student) had students with an average math score of 83.515798. However, there was a slight increase in math scores, 83.599686, in schools with budgets between \$597.25 and \$616.5. 

* Average math and reading scores decreased as school size increased. Schools with the smallest number of students (<1000) had average math scores of 83.821598 and average reading scores of 83.929843. Schools with the greatest number of students (>2000) had average math scores of 77.746417 and average reading scores of 81.344493. 

* Charter schools out-performed district schools across all metrics (average math scores, average reading scores, percent passing math, percent passing reading, and overall passing rate). However, more analysis is required to determine if this result is due to other factors, such as smaller student populations.
---

In [1]:
import pandas as pd
import os

In [2]:
schools_file = os.path.join('Resources', 'schools_complete.csv')
schools_df = pd.read_csv(schools_file)
schools_df.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [3]:
students_file = os.path.join('Resources', 'students_complete.csv')
students_df = pd.read_csv(students_file)
students_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [4]:
schools_students_df = pd.merge(schools_df, students_df, on='school_name')
schools_students_df.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


## District Summary


In [5]:
total_schools = schools_df['School ID'].count()
total_students = students_df['Student ID'].count()
total_budget = schools_df['budget'].sum()

ave_math_score = students_df['math_score'].mean()
ave_reading_score = students_df['reading_score'].mean()

passing_math_df = students_df[students_df['math_score'] >= 70]
num_passing_math = passing_math_df['math_score'].count()
perc_passing_math = (num_passing_math / total_students) * 100

passing_reading_df = students_df[students_df['reading_score'] >= 70]
num_passing_reading = passing_reading_df['reading_score'].count()
perc_passing_reading = (num_passing_reading / total_students) * 100

overall_passing = (perc_passing_math + perc_passing_reading) / 2

In [6]:
total_students = '{:,}'.format(total_students)
total_budget = '${:,.2f}'.format(total_budget)

district_summary_df = pd.DataFrame({
    'Total Schools': total_schools,
    'Total Students': total_students,
    'Total Budget': total_budget, 
    'Ave Math Score': ave_math_score,
    'Ave Reading Score': ave_reading_score,
    '% Passing Math': perc_passing_math,
    '% Passing Reading': perc_passing_reading,
    '% Overall Passing Rate': overall_passing,
}, index=[0])

district_summary_df

,Total Schools,Total Students,Total Budget,Ave Math Score,Ave Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.393158


## School Summary

In [7]:
# from schools_df
school_name_type_s = schools_df.set_index(['school_name'])['type']
school_budget_s = schools_df.set_index(['school_name'])['budget']

In [8]:
# from grouped_schools_students_df
schools_students_gb = schools_students_df.groupby(['school_name'])
total_students_per_school_s = schools_students_gb['Student ID'].count()
ave_math_score_per_school_s = schools_students_gb['math_score'].mean()
ave_reading_score_per_school_s = schools_students_gb['reading_score'].mean()

per_student_budget_s = school_budget_s / total_students_per_school_s

In [9]:
# passing scores per school
passing_math_schools_students_df = schools_students_df[schools_students_df['math_score']>=70]
passing_math_schools_students_gb = passing_math_schools_students_df.groupby(['school_name'])
num_passing_math_per_school_s = passing_math_schools_students_gb['math_score'].count()

passing_reading_schools_students_df = schools_students_df[schools_students_df['reading_score']>=70]
passing_reading_schools_students_gb = passing_reading_schools_students_df.groupby(['school_name'])
num_passing_reading_per_school_s = passing_reading_schools_students_gb['reading_score'].count()

# calculations
perc_passing_math_per_school_s = (num_passing_math_per_school_s / total_students_per_school_s) * 100
perc_passing_reading_per_school_s = (num_passing_reading_per_school_s / total_students_per_school_s) * 100
overall_passing_per_school_s = (perc_passing_math_per_school_s + perc_passing_reading_per_school_s)/2

In [10]:
school_summary_df = pd.DataFrame({
    'School Type': school_name_type_s,
    'Total Students': total_students_per_school_s, 
    'Total School Budget': school_budget_s, 
    'Per Student Budget': per_student_budget_s, 
    'Ave Math Score': ave_math_score_per_school_s, 
    'Ave Reading Score': ave_reading_score_per_school_s, 
    '% Passing Math': perc_passing_math_per_school_s,
    '% Passing Reading': perc_passing_reading_per_school_s, 
    '% Overall Passing Rate': overall_passing_per_school_s
})

school_summary_df['Total Students'] = school_summary_df['Total Students'].map('{:,}'.format)
school_summary_df['Total School Budget'] = school_summary_df['Total School Budget'].map('${:,.2f}'.format)
school_summary_df['Per Student Budget'] = school_summary_df['Per Student Budget'].map('${:,.2f}'.format)

school_summary_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Ave Math Score,Ave Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668


## Top Performing Schools (By Passing Rate)

In [11]:
school_summary_df.sort_values(by='% Overall Passing Rate', ascending=False).head()

,School Type,Total Students,Total School Budget,Per Student Budget,Ave Math Score,Ave Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

In [12]:
school_summary_df.sort_values(by='% Overall Passing Rate').head()

,School Type,Total Students,Total School Budget,Per Student Budget,Ave Math Score,Ave Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## School Math Scores by Grade

In [13]:
# passing_math_schools_students_df = schools_students_df.loc[schools_students_df['grade']=='9th']
# ninth_graders_scores = ninth_graders.groupby(['school_name']).mean()['math_score']

ninth_gr_df = schools_students_df[schools_students_df['grade']=='9th']
tenth_gr_df = schools_students_df[schools_students_df['grade']=='10th']
eleventh_gr_df = schools_students_df[schools_students_df['grade']=='11th']
twelfth_gr_df = schools_students_df[schools_students_df['grade']=='12th']

nineth_gr_ave_math_score_s = ninth_gr_df.groupby('school_name')['math_score'].mean()
tenth_gr_ave_math_score_s = tenth_gr_df.groupby('school_name')['math_score'].mean()
eleventh_gr_ave_math_score_s = eleventh_gr_df.groupby('school_name')['math_score'].mean()
twelfth_gr_ave_math_score_s = twelfth_gr_df.groupby('school_name')['math_score'].mean()

In [14]:
school_math_scores_by_gr_df = pd.DataFrame({
    '9th': nineth_gr_ave_math_score_s,
    '10th': tenth_gr_ave_math_score_s,
    '11th': eleventh_gr_ave_math_score_s,
    '12th': twelfth_gr_ave_math_score_s
})

school_math_scores_by_gr_df.index.name = None

school_math_scores_by_gr_df.head()

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164


## School Reading Scores by Grade

In [15]:
# ninth_graders_scores = ninth_graders.groupby(['school_name']).mean()['math_score']

nineth_gr_ave_reading_score_s = ninth_gr_df.groupby('school_name')['reading_score'].mean()
tenth_gr_ave_reading_score_s = tenth_gr_df.groupby('school_name')['reading_score'].mean()
eleventh_gr_ave_reading_score_s = eleventh_gr_df.groupby('school_name')['reading_score'].mean()
twelfth_gr_ave_reading_score_s = twelfth_gr_df.groupby('school_name')['reading_score'].mean()

In [16]:
school_reading_scores_by_gr_df = pd.DataFrame({
    '9th': nineth_gr_ave_reading_score_s,
    '10th': tenth_gr_ave_reading_score_s,
    '11th': eleventh_gr_ave_reading_score_s,
    '12th': twelfth_gr_ave_reading_score_s
})

school_reading_scores_by_gr_df.index.name = None

school_reading_scores_by_gr_df.head()

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699


## Scores by School Spending

In [17]:
# creating bins of num of data points
max_student_budget = school_summary_df['Per Student Budget'].max().replace('$', '')
min_student_budget = school_summary_df['Per Student Budget'].min().replace('$', '')

total_range = float(max_student_budget) - float(min_student_budget)
category_range = total_range / 4

second_bin = float(min_student_budget) + category_range
third_bin = second_bin + category_range
fourth_bin = third_bin + category_range
fifth_bin = fourth_bin + category_range

In [18]:
first_label = '' + str(min_student_budget) + '-' + str(second_bin) + ''
second_label = '' + str(second_bin) + '-' + str(third_bin) + ''
third_label = '' + str(third_bin) + '-' + str(fourth_bin) + ''
fourth_label = '' + str(fourth_bin) + '-' + str(fifth_bin) + ''

In [19]:
spending_bins = [float(min_student_budget), second_bin, third_bin, fourth_bin, fifth_bin]
spending_labels = [first_label, second_label, third_label, fourth_label]

school_summary_df['Budget Range Per Student'] = pd.cut(per_student_budget_s, spending_bins, labels=spending_labels)

In [20]:
budget_range_gb = school_summary_df.groupby('Budget Range Per Student')

ave_math_score_per_budget_s = budget_range_gb['Ave Math Score'].mean()
ave_reading_score_per_budget_s = budget_range_gb['Ave Reading Score'].mean()
perc_passing_math_per_budget_s = budget_range_gb['% Passing Math'].mean()
perc_passing_reading_per_budget_s = budget_range_gb['% Passing Reading'].mean()
overall_passing_per_budget_s = (perc_passing_math_per_budget_s + perc_passing_reading_per_budget_s)/2

In [21]:
ave_student_spending_budget_df = pd.DataFrame({
    'Ave Math Score': ave_math_score_per_budget_s, 
    'Ave Reading Score': ave_reading_score_per_budget_s, 
    '% Passing Math': perc_passing_math_per_budget_s, 
    '% Passing Reading': perc_passing_reading_per_budget_s, 
    '% Overall Passing Rate': overall_passing_per_budget_s
})

ave_student_spending_budget_df

,Ave Math Score,Ave Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Budget Range Per Student,,,,,
578.00-597.25,83.515798,83.915256,93.324222,96.634622,94.979422
597.25-616.5,83.599686,83.885211,94.230858,95.900287,95.065572
616.5-635.75,80.199966,82.425360,80.036217,89.536122,84.786170
635.75-655.0,77.866721,81.368774,70.347325,82.995575,76.671450


## Scores by School Size

In [22]:
school_size_bins = [0, 1000, 2000, 5000]
school_size_labels = ['Small (<1000)', 'Medium (1000-2000)', 'Large (>2000)']

# add column to school_summary_df so can group by it
school_summary_df['Total Students Range'] = pd.cut(total_students_per_school_s, school_size_bins, labels=school_size_labels)

In [23]:
# size_math_scores = per_school_summary.groupby(['School Size']).mean()['Average Math Score']

school_size_gb = school_summary_df.groupby('Total Students Range')

ave_math_score_per_size_s = school_size_gb['Ave Math Score'].mean()
ave_reading_score_per_size_s = school_size_gb['Ave Reading Score'].mean()
perc_passing_math_per_size_s = school_size_gb['% Passing Math'].mean()
perc_passing_reading_per_size_s = school_size_gb['% Passing Reading'].mean()
overall_passing_per_size_s = (perc_passing_math_per_size_s + perc_passing_reading_per_size_s)/2

In [24]:
ave_school_size_summary_df = pd.DataFrame({
    'Ave Math Score': ave_math_score_per_size_s, 
    'Ave Reading Score': ave_reading_score_per_size_s, 
    '% Passing Math': perc_passing_math_per_size_s, 
    '% Passing Reading': perc_passing_reading_per_size_s, 
    '% Overall Passing Rate': overall_passing_per_size_s, 
})

ave_school_size_summary_df

,Ave Math Score,Ave Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Total Students Range,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (>2000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

In [25]:
# type_math_scores = per_school_summary.groupby(['School Type']).mean()['Average Math Score']

school_type_gb = school_summary_df.groupby('School Type')

ave_math_score_by_type_s = school_type_gb['Ave Math Score'].mean()
ave_reading_score_by_type_s = school_type_gb['Ave Reading Score'].mean()
perc_passing_math_by_type_s = school_type_gb['% Passing Math'].mean()
perc_passing_reading_by_type_s = school_type_gb['% Passing Reading'].mean()
overall_passing_by_type_s = school_type_gb['% Overall Passing Rate'].mean()

In [26]:
school_type_summary_df = pd.DataFrame({
    'Ave Math Score': ave_math_score_by_type_s,
    'Ave Reading Score': ave_reading_score_by_type_s,
    '% Passing Math': perc_passing_math_by_type_s,
    '% Passing Reading': perc_passing_reading_by_type_s,
    '% Overall Passing Rate': overall_passing_by_type_s
})

school_type_summary_df

,Ave Math Score,Ave Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
